In [1]:
%matplotlib qt
from bluesky import RunEngine
from bluesky.plan_stubs import kickoff, complete, collect, \
    open_run, close_run, mv
from ophyd import Device
from ophyd.flyers import FlyerInterface
from ophyd.status import SubscriptionStatus
from ophyd.sim import motor1
from matplotlib.pyplot import ion
from bluesky.utils import install_nb_kicker
install_nb_kicker()

ion()
RE = RunEngine()

## Fly Scan

In [2]:
class FlyerDevice(FlyerInterface, Device):
    
    def kickoff(self):
        print("Start asynchronous task")
        return motor1.set(3)

    def fly_monitor(self, **kwargs):
        position = kwargs["obj"].readback.get()
        print(f"Monitoring: {position}")
        if position == 3:
            return True
        return False

    def complete(self):
        return SubscriptionStatus(motor1, callback=self.fly_monitor)
    
    def describe_collect(self):
        return {"decription": {}}
    
    def collect(self):
        print("Collecting data after scan")
        return [{"data": {}, "timestamps": {}, "time": 0, "seq_num": 0}]

In [3]:
def flyscan(flyer):
    yield from open_run()
    yield from kickoff(flyer, wait=True)
    yield from complete(flyer, wait=True)
    yield from collect(flyer)
    yield from close_run()
    
flyer_device = FlyerDevice(name="flyer_device")
RE(mv(motor1, 5))
RE(flyscan(flyer_device))

Start asynchronous task
Monitoring: 3.0


('bf336d4c-facd-452e-99b4-674a123a8afe',)

In [4]:
Standard plans: per_step


Combinations of the above that are often convenient:

trigger_and_read(devices[, name])
	

Trigger and read a list of detectors and bundle readings into one Event.

one_1d_step(detectors, motor, step[, ...])
	

Inner loop of a 1D step scan

one_nd_step(detectors, step, pos_cache[, ...])
	

Inner loop of an N-dimensional step scan

one_shot(detectors[, take_reading])
	

Inner loop of a count.

move_per_step(step, pos_cache)
	

Inner loop of an N-dimensional step scan without any readings

SyntaxError: invalid syntax (2366441638.py, line 1)